# Inicio de modelo Tree

## General modelo

In [15]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn import tree
import seaborn as sns

from sklearn.preprocessing import scale
import sklearn

In [16]:
df = pd.read_csv("train_titanic.csv")

## Preprocesamiento de dt

In [17]:

df2 = df.copy()
df2 = df2[['Survived','Pclass','Sex','Age','Fare']]
df2['Age'] = df2['Age'].fillna(df2['Age'].mean())
df2.info()
df2.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   Fare      891 non-null    float64
dtypes: float64(2), int64(2), object(1)
memory usage: 34.9+ KB


,Survived,Pclass,Age,Fare
count,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,32.204208
std,0.486592,0.836071,13.002015,49.693429
min,0.000000,1.000000,0.420000,0.000000
25%,0.000000,2.000000,22.000000,7.910400
50%,0.000000,3.000000,29.699118,14.454200
75%,1.000000,3.000000,35.000000,31.000000
max,1.000000,3.000000,80.000000,512.329200


In [18]:
df2['Pclass'] = df2['Pclass'].astype(str)
df2['Sex'] = df2['Sex'].astype(str)

df2 = pd.get_dummies(df2, drop_first=True)

In [19]:
df2[['Pclass_2','Pclass_3', 'Sex_male']] = df2[['Pclass_2','Pclass_3', 'Sex_male']].astype(int)
df2

,Survived,Age,Fare,Pclass_2,Pclass_3,Sex_male
0,0,22.000000,7.2500,0,1,1
1,1,38.000000,71.2833,0,0,0
2,1,26.000000,7.9250,0,1,0
3,1,35.000000,53.1000,0,0,0
4,0,35.000000,8.0500,0,1,1
...,...,...,...,...,...,...
886,0,27.000000,13.0000,1,0,1
887,1,19.000000,30.0000,0,0,0
888,0,29.699118,23.4500,0,1,0
889,1,26.000000,30.0000,0,0,1


## Creacion de modelo

In [20]:
from sklearn.tree import DecisionTreeClassifier # predict roba
from sklearn.tree import DecisionTreeRegressor 

In [21]:
# tree normal, sin modificaciones

X=df2.drop('Survived',axis=1)
y=df2['Survived'] # 0 o 1

# aqui construimos un modelo
model = DecisionTreeClassifier()
#(max_depth=10)
# cada modelo tiene una función que dice que parametros tiene
# estos serian los parametros por defecto
print(model.get_params())


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50)
tree_model=model.fit(X_train,y_train)

# train
y_pred=tree_model.predict(X_train)
probs=tree_model.predict_proba(X_train)
print("Train AUC", sklearn.metrics.roc_auc_score(y_train, probs[:,1]))
print("Train Accuracy", sklearn.metrics.accuracy_score(y_train,y_pred))

# test
y_pred=tree_model.predict(X_test)
probs=tree_model.predict_proba(X_test)
print("Test AUC", sklearn.metrics.roc_auc_score(y_test, probs[:,1]))
print("Test Accuracy",  sklearn.metrics.accuracy_score(y_test,y_pred))

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}
Train AUC 0.99988624988625
Train Accuracy 0.9932584269662922
Test AUC 0.7616388479790541
Test Accuracy 0.7645739910313901


In [22]:
list(X.columns)

['Age', 'Fare', 'Pclass_2', 'Pclass_3', 'Sex_male']

### Dibujo de modelo

In [23]:
print(tree.export_graphviz(tree_model, feature_names = list(X.columns)) )

with open("classifier.txt", "w") as f:
    f = tree.export_graphviz(tree_model, out_file=f,  feature_names = list(X.columns))

digraph Tree {
node [shape=box, fontname="helvetica"] ;
edge [fontname="helvetica"] ;
0 [label="Sex_male <= 0.5\ngini = 0.444\nsamples = 445\nvalue = [297, 148]"] ;
1 [label="Pclass_3 <= 0.5\ngini = 0.419\nsamples = 144\nvalue = [43, 101]"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="Age <= 2.5\ngini = 0.117\nsamples = 80\nvalue = [5, 75]"] ;
1 -> 2 ;
3 [label="Fare <= 88.775\ngini = 0.5\nsamples = 2\nvalue = [1, 1]"] ;
2 -> 3 ;
4 [label="gini = 0.0\nsamples = 1\nvalue = [0, 1]"] ;
3 -> 4 ;
5 [label="gini = 0.0\nsamples = 1\nvalue = [1, 0]"] ;
3 -> 5 ;
6 [label="Fare <= 26.125\ngini = 0.097\nsamples = 78\nvalue = [4, 74]"] ;
2 -> 6 ;
7 [label="Age <= 27.5\ngini = 0.227\nsamples = 23\nvalue = [3, 20]"] ;
6 -> 7 ;
8 [label="Fare <= 19.875\ngini = 0.408\nsamples = 7\nvalue = [2, 5]"] ;
7 -> 8 ;
9 [label="Age <= 23.5\ngini = 0.278\nsamples = 6\nvalue = [1, 5]"] ;
8 -> 9 ;
10 [label="gini = 0.0\nsamples = 3\nvalue = [0, 3]"] ;
9 -> 10 ;
11 [label="Fare <= 15.87

https://dreampuf.github.io/GraphvizOnline/

### modificaciones de modelo, afinarlo

In [24]:
# =============================================================================
# Building a tree decision model
# =============================================================================
# especificando el modelo
model = DecisionTreeClassifier()

# vemos los parametros que puede tener
print(model.get_params())
params={'max_depth': [7],# Maxima pofundidad del arbol #5
        'max_leaf_nodes': [10,15,20], # maximo de hojas del arbol #3
        'min_impurity_decrease' : [0.01, 0.02],#2 # un nuevo nodo se harà si al hacerse se decrece la impurity en un threshold por encima del valor
        'min_samples_split': [10,20], ##4 The minimum number of samples required to split an internal node:
        'ccp_alpha': [0.0,0.01]#2 #240
        }

# scoring: lista de metricas a obtener
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
scoring = ['accuracy', 'roc_auc']

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}


### busqueda del mejor y medias

In [25]:
# CV (cross validación)
n_cv=8 # 5 folds
# creamos la GridSeacrh # cross-validation
grid_solver = GridSearchCV(estimator = model, # model to train
                   param_grid = params, # param_grid
                   scoring = scoring,
                   cv = n_cv,
                   refit = 'roc_auc', # cuando tienes mas de un score para tener un criterio de cual es el mejor modelo
                   verbose = 2)



model_result = grid_solver.fit(X_train,y_train)
# hemos ajustado el modelo

# best score es la media del best estimator
print("Mejor score:", model_result.best_score_)
# tenemos los mejores parametros
print("Mejores parametros:",model_result.best_params_)

# Train
y_pred=model_result.predict(X_train)
probs=model_result.predict_proba(X_train)
print("Train AUC:", sklearn.metrics.roc_auc_score(y_train,probs[:,1]))
print("Train Accuracy:", sklearn.metrics.accuracy_score(y_train,y_pred))

# test
model_result.score(X_test,y_test)
y_pred=model_result.predict(X_test)
probs=model_result.predict_proba(X_test)
print("Test AUC:", sklearn.metrics.roc_auc_score(y_test,probs[:,1]))
print("Test Accuracy:", sklearn.metrics.accuracy_score(y_test,y_pred))

Fitting 8 folds for each of 24 candidates, totalling 192 fits
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.01, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.01, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.01, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.01, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.01, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.01, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.01, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_imp

[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.02, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.02, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.02, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.02, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.02, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.02, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.02, min_samples_split=10; total time=   0.0s
[CV] END ccp_alpha=0.0, max_depth=7, max_leaf_nodes=10, min_impurity_decrease=0.02, min_samples_split=10; total time=   0.0s


### Creacion del mejor modelo

In [26]:
# tenemos un atributo del mejor modelo
best_model=model_result.best_estimator_


# nos quedamos con el mejor modelo y reestimamos con toda la muestra
final_model=best_model.fit(X,y)

In [27]:
best_model

DecisionTreeClassifier(max_depth=7, max_leaf_nodes=10,
                       min_impurity_decrease=0.01, min_samples_split=10)

In [28]:
print(tree.export_graphviz(final_model, feature_names = list(X.columns)) )

with open("classifier.txt", "w") as f:
    f = tree.export_graphviz(final_model, out_file=f,  feature_names = list(X.columns))

digraph Tree {
node [shape=box, fontname="helvetica"] ;
edge [fontname="helvetica"] ;
0 [label="Sex_male <= 0.5\ngini = 0.473\nsamples = 891\nvalue = [549, 342]"] ;
1 [label="Pclass_3 <= 0.5\ngini = 0.383\nsamples = 314\nvalue = [81, 233]"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
3 [label="gini = 0.1\nsamples = 170\nvalue = [9, 161]"] ;
1 -> 3 ;
4 [label="Fare <= 23.35\ngini = 0.5\nsamples = 144\nvalue = [72, 72]"] ;
1 -> 4 ;
9 [label="gini = 0.484\nsamples = 117\nvalue = [48, 69]"] ;
4 -> 9 ;
10 [label="gini = 0.198\nsamples = 27\nvalue = [24, 3]"] ;
4 -> 10 ;
2 [label="Age <= 6.5\ngini = 0.306\nsamples = 577\nvalue = [468, 109]"] ;
0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;
5 [label="gini = 0.444\nsamples = 24\nvalue = [8, 16]"] ;
2 -> 5 ;
6 [label="Fare <= 26.269\ngini = 0.28\nsamples = 553\nvalue = [460, 93]"] ;
2 -> 6 ;
7 [label="gini = 0.194\nsamples = 404\nvalue = [360, 44]"] ;
6 -> 7 ;
8 [label="gini = 0.441\nsamples = 149\nvalue = [